In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [6]:
import os
import shutil
import random

# Set your dataset directory path
dataset_dir = "/content/drive/MyDrive/Food_Classification_India"

# Create directories for train, validation, and test sets
train_dir = "/content/train"
validation_dir = "/content/validate"
test_dir = "/content/test"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)


In [7]:
# Define the split ratio
train_split = 0.7
val_split = 0.15
test_split = 0.15

In [8]:
# Iterate through each folder in the dataset
for folder_name in os.listdir(dataset_dir):
    folder_path = os.path.join(dataset_dir, folder_name)

    # Get list of images in the folder
    images = os.listdir(folder_path)
    random.shuffle(images)  # Shuffle images

    # Calculate split sizes
    num_images = len(images)
    num_train = int(num_images * train_split)
    num_val = int(num_images * val_split)
    num_test = num_images - num_train - num_val

    # Split images
    train_images = images[:num_train]
    val_images = images[num_train:num_train + num_val]
    test_images = images[num_train + num_val:]

    # Move images to respective directories
    for img in train_images:
        src = os.path.join(folder_path, img)
        dst = os.path.join(train_dir, folder_name)
        os.makedirs(dst, exist_ok=True)
        shutil.copy(src, dst)

    for img in val_images:
        src = os.path.join(folder_path, img)
        dst = os.path.join(validation_dir, folder_name)
        os.makedirs(dst, exist_ok=True)
        shutil.copy(src, dst)

    for img in test_images:
        src = os.path.join(folder_path, img)
        dst = os.path.join(test_dir, folder_name)
        os.makedirs(dst, exist_ok=True)
        shutil.copy(src, dst)

print("Dataset split complete.")

Dataset split complete.


In [9]:
img_height, img_width = 224, 224
batch_size = 10

train_ds = tf.keras.utils.image_dataset_from_directory(
    "train",
    image_size = (img_height, img_width),
    batch_size = batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "validate",
    image_size = (img_height, img_width),
    batch_size = batch_size
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "test",
    image_size = (img_height, img_width),
    batch_size = batch_size
)

Found 2751 files belonging to 20 classes.
Found 588 files belonging to 20 classes.
Found 590 files belonging to 20 classes.


In [10]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Rescaling(1./255),
     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),
     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),
     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(128, activation="relu"),
     tf.keras.layers.Dense(56, activation="softmax")  # Adjusted for 20 classes
    ]
)


In [11]:
model.compile(
    optimizer="adam",
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics=['accuracy']
)

In [12]:
model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 5
)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


276/276 [==============================] - 233s 835ms/step - loss: 2.8743 - accuracy: 0.1170 - val_loss: 2.6550 - val_accuracy: 0.1497
Epoch 2/5
276/276 [==============================] - 220s 797ms/step - loss: 2.3902 - accuracy: 0.2563 - val_loss: 2.5876 - val_accuracy: 0.1973
Epoch 3/5
276/276 [==============================] - 219s 792ms/step - loss: 1.8323 - accuracy: 0.4329 - val_loss: 2.7700 - val_accuracy: 0.2534
Epoch 4/5
276/276 [==============================] - 226s 817ms/step - loss: 1.0360 - accuracy: 0.6685 - val_loss: 3.4788 - val_accuracy: 0.2551
Epoch 5/5
276/276 [==============================] - 223s 807ms/step - loss: 0.4507 - accuracy: 0.8684 - val_loss: 4.5275 - val_accuracy: 0.2296


In [13]:
model.evaluate(train_ds)

276/276 [==============================] - 88s 313ms/step - loss: 0.3025 - accuracy: 0.9062


[0.30245086550712585, 0.9062159061431885]

In [14]:
import os

# Get the class names from the folder names in the train directory
class_names = sorted(os.listdir(train_dir))

# Print the class names to verify
print(class_names)


['burger', 'butter_naan', 'chai', 'chapati', 'chole_bhature', 'dal_makhani', 'dhokla', 'fried_rice', 'idli', 'jalebi', 'kaathi_rolls', 'kadai_paneer', 'kulfi', 'masala_dosa', 'momos', 'paani_puri', 'pakode', 'pav_bhaji', 'pizza', 'samosa']


In [15]:
# Resize images to the correct input shape
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, (224, 224)), y))
val_ds = val_ds.map(lambda x, y: (tf.image.resize(x, (224, 224)), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, (224, 224)), y))


In [16]:
import tensorflow as tf

# Load pre-trained VGG16 model without the top dense layers
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [17]:
# Freeze the pre-trained layers
base_model.trainable = False

# Add new dense layers on top of the pre-trained model
validate_synchronization_aggregation_trainablemodel = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

In [18]:
# Compile the model
validate_synchronization_aggregation_trainablemodel.compile(optimizer='adam',
                                                           loss='sparse_categorical_crossentropy',
                                                           metrics=['accuracy'])



In [20]:
# Train the model
history = validate_synchronization_aggregation_trainablemodel.fit(train_ds,
                                                                  validation_data=val_ds,
                                                                  epochs=5)

Epoch 1/5
276/276 [==============================] - 1859s 7s/step - loss: 3.9506 - accuracy: 0.2196 - val_loss: 2.1282 - val_accuracy: 0.3673
Epoch 2/5
276/276 [==============================] - 1844s 7s/step - loss: 2.1280 - accuracy: 0.3828 - val_loss: 1.7397 - val_accuracy: 0.4779
Epoch 3/5
276/276 [==============================] - 1893s 7s/step - loss: 1.7579 - accuracy: 0.4875 - val_loss: 1.4798 - val_accuracy: 0.5595
Epoch 4/5
276/276 [==============================] - 1832s 7s/step - loss: 1.4361 - accuracy: 0.5678 - val_loss: 1.3998 - val_accuracy: 0.6054
Epoch 5/5
276/276 [==============================] - 1822s 7s/step - loss: 1.3285 - accuracy: 0.5932 - val_loss: 1.3209 - val_accuracy: 0.6259


In [21]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [22]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = validate_synchronization_aggregation_trainablemodel.evaluate(test_ds)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)

59/59 [==============================] - 330s 5s/step - loss: 1.2409 - accuracy: 0.6288
Test Accuracy: 0.6288135647773743


In [23]:
converter = tf.lite.TFLiteConverter.from_keras_model(validate_synchronization_aggregation_trainablemodel)
tflite_model = converter.convert()

with open("model.tflite", 'wb') as f:
  f.write(tflite_model)